In [1]:
import nucle
# Path to the file you want to process
file_path_a3 = 'ABC.train.gold.bea19.m2'
file_path_a4 = 'ABCN.dev.gold.bea19.m2'

# Generate examples from the file
nuc = nucle.NUCLE()
examples_a3 = list(nuc._generate_examples(file_path_a3))
examples_a4 = list(nuc._generate_examples(file_path_a4))

/Users/frankli/miniconda3/envs/final_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import re

def detokenize(tokens):
    sentence = ""
    open_double_quote = True
    open_single_quote = True
    no_space_tokens = ["'s", "n't", "'S", "'m", "'ll", "'ve", "'d", "'re", "--", ".-", "%"]
    prev_no_space_tokens = ["'", "$", "-", '"', ' ', '£', '(', '[', "{"]
    
    for token in tokens:
        # Handle quotes
        if token == '"':
            if open_double_quote:
                sentence += ' "' if sentence and not sentence[-1].isspace() else '"'
                open_double_quote = False
            else:
                sentence = sentence.rstrip() + '"'
                open_double_quote = True
        elif token == "'":
            if open_single_quote:
                sentence += " '" if sentence and not sentence[-1].isspace() else "'"
                open_single_quote = False
            else:
                sentence = sentence.rstrip() + "'"
                open_single_quote = True
        # Handle punctuation marks
        elif token in {".", ",", "!", "?", ":", ";", "-"}:
            sentence = sentence.rstrip() + token
        # Handle closing brackets
        elif token in {")", "]", "}"}:
            sentence = sentence.rstrip() + token
        # Handle opening brackets
        elif token in {"(", "[", "{"}:
            sentence += token
        # Handle special cases for contractions and possessives
        elif token in no_space_tokens:
            sentence += token
        else:
            # General case for regular tokens
            if sentence and sentence[-1] not in prev_no_space_tokens:
                sentence += " "
            sentence += token
    
    # Clean up spacing and return
    return sentence.strip()

# Example usage
tokens = ['Yesterday', ',', 'I', 'had', 'a', 'wonderful', 'day', '.', 'In', 'the', 'morning', 'I', 'went', 'to', 'a', 'Galician', 'basketball', 'training', 'with', 'a', 'coach', 'of', 'Spanish', 'selection', ',', 'when', 'finished', 'went', 'to', 'my', 'parents', 'to', 'a', 'hotel', 'to', 'eat', 'and', 'stay', 'with', 'some', 'friends', ',', 'after', 'the', 'food', 'meet', 'with', 'friend', "'s", 'of', 'my', 'parents', 'and', 'went', 'to', 'a', 'theme', 'park', 'and', 'we', 'rode', 'to', 'a', 'rollers', 'coaster', ',', 'it', 'was', 'very', 'exciting', 'and', 'we', 'ate', 'a', 'chocolate', 'ice', 'cream', ',', 'it', 'was', 'delicious', '.', 'At', 'night', 'we', 'went', 'to', 'a', 'match', 'of', 'Spanish', 'basketball', 'league', ',', 'Obradoiro', 'versus', 'Real', 'Madrid', 'and', 'that', 'match', 'won', 'the', 'team', 'of', 'Real', 'Madrid', 'and', 'then', 'I', 'went', 'home', 'to', 'sleep', '.']
sentence = detokenize(tokens)
print(sentence)


Yesterday, I had a wonderful day. In the morning I went to a Galician basketball training with a coach of Spanish selection, when finished went to my parents to a hotel to eat and stay with some friends, after the food meet with friend's of my parents and went to a theme park and we rode to a rollers coaster, it was very exciting and we ate a chocolate ice cream, it was delicious. At night we went to a match of Spanish basketball league, Obradoiro versus Real Madrid and that match won the team of Real Madrid and then I went home to sleep.


In [3]:
import pandas as pd
all_ds = [examples_a3, examples_a4]
all_names = ["examples_a3", "examples_a4"]
tokens = set()
import nltk
from nltk.tokenize import TreebankWordTokenizer, TreebankWordDetokenizer
detokenizer = TreebankWordDetokenizer()
for ds, name in zip(all_ds, all_names):
    before_statements = []
    after_statements = []
    corrections = []
    for indexed_example in ds:
        example = indexed_example[1]
        before = example['src_tokens']
        after = example['tgt_tokens']
        correct = example['corrections']
        # for token in before:
        #     if len(token) < 4 and token not in tokens and ("'" in token or "." in token or "-" in token or "-" in token or "*" in token or '£' in token):
        #         tokens.add(token)
        #         print(token)
        #         print(before)
        #         print(detokenize(before))
        before_statements.append(detokenizer.detokenize(before))
        after_statements.append(detokenizer.detokenize(after))
        corrections.append(correct)
    data = {
        "Before Statements": before_statements,
        "After Statements": after_statements,
        "Error Correction": corrections
    }
    df = pd.DataFrame(data)
    df.to_csv(f"../../csv_data/{name}.csv", index = False)

In [2]:
!pwd

/Users/frankli/Desktop/Yale Classes/Classes 2024/Ling/final/ling-final-project/wi+locness/m2
